In [30]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.stats.stats import pearsonr
import pickle
import statistics
%matplotlib inline

In [31]:
cdk_desc = pd.read_csv(r'data_TA/cdk_desc.csv')

cdk_desc

,Title,ALogP,ALogp2,AMR,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,Mol1,0.2908,0.084565,75.7828,11.994344,19.000412,-0.326879,0.204396,3.983285,9.371560,...,315.155847,43.839390,1.992700,19.045171,2.547340,8.940355,1161,34,1.056,108
1,Mol2,1.7599,3.097248,106.0192,11.850000,19.000417,-0.326851,0.233856,3.984015,9.866476,...,419.182062,60.422658,2.014089,22.109715,5.134918,9.417215,2841,49,2.801,152
2,Mol3,0.0863,0.007448,85.3487,11.988451,19.000414,-0.360020,0.278314,3.983783,9.619386,...,357.166412,49.540696,1.981628,21.837220,4.969081,9.310598,1683,40,1.084,124
3,Mol4,1.5280,2.334784,111.7861,11.996640,19.000419,-0.335855,0.233992,3.984031,9.937788,...,463.171891,67.174803,2.035600,28.277686,11.293906,9.426195,3706,55,2.290,174
4,Mol5,0.6093,0.371246,103.8623,11.850000,19.000417,-0.326858,0.235369,3.984004,9.824791,...,420.177311,60.422658,2.014089,25.115436,5.134918,12.422936,2841,49,1.549,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Mol130,2.4573,6.038323,128.9326,11.850000,31.973074,-0.326855,0.202569,4.563459,14.040421,...,501.155612,68.979139,2.028798,25.316101,7.476418,8.774386,3596,54,2.906,182
130,Mol131,1.1899,1.415862,117.8384,11.850000,31.972073,-0.326866,0.204219,3.984556,12.441325,...,508.139211,70.801056,2.022887,33.473601,10.634027,11.848343,3997,56,3.788,188
131,Mol132,1.3954,1.947141,118.0548,11.850000,31.972073,-0.326871,0.204315,3.984551,12.439904,...,526.129789,72.658808,2.018300,36.005554,10.634853,11.849681,4318,58,3.949,194
132,Mol133,1.1735,1.377102,124.3016,11.996133,31.972073,-0.377504,0.204290,3.984572,12.458502,...,538.149776,74.682741,2.018452,36.274626,13.427469,11.855833,4675,60,3.703,198


In [32]:
mordred_desc = pd.read_csv(r'data_TA/mordred_desc.csv')

mordred_desc

,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,all.0,16.689085,14.095451,0,2,27.046434,2.344744,4.688370,27.046434,1.229383,...,9.742556,63.170473,315.155847,7.503711,1161,34,108.0,122.0,8.277778,4.916667
1,all.1,23.081555,17.973236,0,1,37.886845,2.351817,4.695720,37.886845,1.262895,...,10.147022,73.120154,419.182062,7.762631,2841,49,152.0,175.0,10.611111,6.638889
2,all.2,18.988744,15.744654,0,1,30.254083,2.346709,4.690436,30.254083,1.210163,...,9.910612,67.041400,357.166412,7.599285,1683,40,124.0,141.0,10.250000,5.527778
3,all.3,25.869542,19.775723,0,1,42.065427,2.405221,4.750609,42.065427,1.274710,...,10.344738,83.014130,463.171891,8.125823,3706,55,174.0,204.0,11.083333,7.166667
4,all.4,23.081555,17.973236,0,1,37.886845,2.351817,4.695720,37.886845,1.262895,...,10.147022,73.120154,420.177311,7.927874,2841,49,152.0,175.0,10.611111,6.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,all.129,26.735417,20.969839,0,1,43.239697,2.514698,4.966567,43.239697,1.271756,...,10.509578,84.521992,501.155612,8.083155,3596,54,182.0,213.0,11.513889,7.291667
130,all.130,27.551914,21.508012,0,1,43.983180,2.496629,4.948414,43.983180,1.256662,...,10.548940,85.349833,508.139211,8.761021,3997,56,188.0,220.0,12.375000,7.458333
131,all.131,28.368411,22.103382,0,1,44.701006,2.498063,4.950099,44.701006,1.241695,...,10.586559,86.517101,526.129789,9.071203,4318,58,194.0,227.0,13.236111,7.625000
132,all.132,28.966128,22.530137,0,1,46.406238,2.498477,4.950597,46.406238,1.254223,...,10.602890,87.612019,538.149776,8.679835,4675,60,198.0,232.0,13.486111,7.958333


In [33]:
padel_desc = pd.read_csv(r'data_TA/padel_desc.csv')

padel_desc

,Name,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,AUTOGEN_all_1,0,-1.0627,1.129331,79.6556,45.508860,6,6,42,22,...,0.639891,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728
1,AUTOGEN_all_2,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,...,0.806407,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973
2,AUTOGEN_all_3,0,-1.2672,1.605796,89.2215,51.164446,6,6,47,25,...,0.843269,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546
3,AUTOGEN_all_4,0,-1.2656,1.601743,119.6603,64.662032,12,12,57,33,...,0.815629,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503
4,AUTOGEN_all_5,0,-1.6474,2.713927,110.0384,59.971239,12,12,53,30,...,0.800274,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,AUTOGEN_all_130,0,0.3636,0.132205,138.9661,74.733204,17,18,62,34,...,0.767598,0.152839,0.455611,0.377196,0.437523,29.375999,164.436509,430.436141,0.651397,1.270330
130,AUTOGEN_all_131,0,-0.3540,0.125316,124.2705,67.995239,17,18,58,35,...,0.574784,0.364984,0.561717,0.357159,0.480373,24.895532,165.105971,384.973632,0.409651,1.399248
131,AUTOGEN_all_132,0,-0.4442,0.197314,124.3983,67.885446,17,18,58,36,...,0.570497,0.368969,0.550420,0.363275,0.480499,24.939796,166.299510,388.900703,0.409199,1.394195
132,AUTOGEN_all_133,0,-0.8526,0.726927,130.9140,71.890825,17,18,62,37,...,0.576007,0.336061,0.533630,0.431886,0.421620,22.937123,144.035418,372.377249,0.368102,1.387136


In [34]:
pIC50_target = pd.read_excel(r'data_TA/pIC50_target.xlsx', index_col=0)

pIC50_target

,pIC50
No.,
1,5.842
2,6.292
3,5.469
4,6.310
5,6.387
...,...
130,7.142
131,7.026
132,7.259


In [35]:
#kolom yang dihapus
cdk_desc = cdk_desc.drop(["Title"], axis=1)

cdk_desc

,ALogP,ALogp2,AMR,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,PPSA-1,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0.2908,0.084565,75.7828,11.994344,19.000412,-0.326879,0.204396,3.983285,9.371560,356.240964,...,315.155847,43.839390,1.992700,19.045171,2.547340,8.940355,1161,34,1.056,108
1,1.7599,3.097248,106.0192,11.850000,19.000417,-0.326851,0.233856,3.984015,9.866476,482.343937,...,419.182062,60.422658,2.014089,22.109715,5.134918,9.417215,2841,49,2.801,152
2,0.0863,0.007448,85.3487,11.988451,19.000414,-0.360020,0.278314,3.983783,9.619386,512.582748,...,357.166412,49.540696,1.981628,21.837220,4.969081,9.310598,1683,40,1.084,124
3,1.5280,2.334784,111.7861,11.996640,19.000419,-0.335855,0.233992,3.984031,9.937788,498.914595,...,463.171891,67.174803,2.035600,28.277686,11.293906,9.426195,3706,55,2.290,174
4,0.6093,0.371246,103.8623,11.850000,19.000417,-0.326858,0.235369,3.984004,9.824791,496.385973,...,420.177311,60.422658,2.014089,25.115436,5.134918,12.422936,2841,49,1.549,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2.4573,6.038323,128.9326,11.850000,31.973074,-0.326855,0.202569,4.563459,14.040421,497.131970,...,501.155612,68.979139,2.028798,25.316101,7.476418,8.774386,3596,54,2.906,182
130,1.1899,1.415862,117.8384,11.850000,31.972073,-0.326866,0.204219,3.984556,12.441325,554.079784,...,508.139211,70.801056,2.022887,33.473601,10.634027,11.848343,3997,56,3.788,188
131,1.3954,1.947141,118.0548,11.850000,31.972073,-0.326871,0.204315,3.984551,12.439904,464.821377,...,526.129789,72.658808,2.018300,36.005554,10.634853,11.849681,4318,58,3.949,194
132,1.1735,1.377102,124.3016,11.996133,31.972073,-0.377504,0.204290,3.984572,12.458502,589.900158,...,538.149776,74.682741,2.018452,36.274626,13.427469,11.855833,4675,60,3.703,198


In [36]:
#kolom yang dihapus
mordred_desc = mordred_desc.drop(["name"], axis=1)

mordred_desc

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,16.689085,14.095451,0,2,27.046434,2.344744,4.688370,27.046434,1.229383,3.992670,...,9.742556,63.170473,315.155847,7.503711,1161,34,108.0,122.0,8.277778,4.916667
1,23.081555,17.973236,0,1,37.886845,2.351817,4.695720,37.886845,1.262895,4.312381,...,10.147022,73.120154,419.182062,7.762631,2841,49,152.0,175.0,10.611111,6.638889
2,18.988744,15.744654,0,1,30.254083,2.346709,4.690436,30.254083,1.210163,4.118618,...,9.910612,67.041400,357.166412,7.599285,1683,40,124.0,141.0,10.250000,5.527778
3,25.869542,19.775723,0,1,42.065427,2.405221,4.750609,42.065427,1.274710,4.424867,...,10.344738,83.014130,463.171891,8.125823,3706,55,174.0,204.0,11.083333,7.166667
4,23.081555,17.973236,0,1,37.886845,2.351817,4.695720,37.886845,1.262895,4.312381,...,10.147022,73.120154,420.177311,7.927874,2841,49,152.0,175.0,10.611111,6.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,26.735417,20.969839,0,1,43.239697,2.514698,4.966567,43.239697,1.271756,4.457359,...,10.509578,84.521992,501.155612,8.083155,3596,54,182.0,213.0,11.513889,7.291667
130,27.551914,21.508012,0,1,43.983180,2.496629,4.948414,43.983180,1.256662,4.485814,...,10.548940,85.349833,508.139211,8.761021,3997,56,188.0,220.0,12.375000,7.458333
131,28.368411,22.103382,0,1,44.701006,2.498063,4.950099,44.701006,1.241695,4.513519,...,10.586559,86.517101,526.129789,9.071203,4318,58,194.0,227.0,13.236111,7.625000
132,28.966128,22.530137,0,1,46.406238,2.498477,4.950597,46.406238,1.254223,4.538301,...,10.602890,87.612019,538.149776,8.679835,4675,60,198.0,232.0,13.486111,7.958333


In [37]:
#kolom yang dihapus
padel_desc = padel_desc.drop(["Name"], axis=1)

padel_desc

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,0,-1.0627,1.129331,79.6556,45.508860,6,6,42,22,20,...,0.639891,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728
1,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,24,...,0.806407,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973
2,0,-1.2672,1.605796,89.2215,51.164446,6,6,47,25,22,...,0.843269,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546
3,0,-1.2656,1.601743,119.6603,64.662032,12,12,57,33,24,...,0.815629,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503
4,0,-1.6474,2.713927,110.0384,59.971239,12,12,53,30,23,...,0.800274,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0,0.3636,0.132205,138.9661,74.733204,17,18,62,34,28,...,0.767598,0.152839,0.455611,0.377196,0.437523,29.375999,164.436509,430.436141,0.651397,1.270330
130,0,-0.3540,0.125316,124.2705,67.995239,17,18,58,35,23,...,0.574784,0.364984,0.561717,0.357159,0.480373,24.895532,165.105971,384.973632,0.409651,1.399248
131,0,-0.4442,0.197314,124.3983,67.885446,17,18,58,36,22,...,0.570497,0.368969,0.550420,0.363275,0.480499,24.939796,166.299510,388.900703,0.409199,1.394195
132,0,-0.8526,0.726927,130.9140,71.890825,17,18,62,37,25,...,0.576007,0.336061,0.533630,0.431886,0.421620,22.937123,144.035418,372.377249,0.368102,1.387136


In [38]:
#satuin semua data
dataset = cdk_desc

for column in mordred_desc:
    dataset[column] = mordred_desc[column]

for column in padel_desc:
    dataset[column] = padel_desc[column]
    
# for column in pIC50_target:
#     dataset[column] = pIC50_target[column]

dataset

,ALogP,ALogp2,AMR,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,PPSA-1,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,-1.0627,1.129331,79.6556,11.994344,19.000412,-0.326879,0.204396,3.983285,9.371560,356.240964,...,0.639891,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728
1,-0.0693,0.004802,113.5328,11.850000,19.000417,-0.326851,0.233856,3.984015,9.866476,482.343937,...,0.806407,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973
2,-1.2672,1.605796,89.2215,11.988451,19.000414,-0.360020,0.278314,3.983783,9.619386,512.582748,...,0.843269,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546
3,-1.2656,1.601743,119.6603,11.996640,19.000419,-0.335855,0.233992,3.984031,9.937788,498.914595,...,0.815629,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503
4,-1.6474,2.713927,110.0384,11.850000,19.000417,-0.326858,0.235369,3.984004,9.824791,496.385973,...,0.800274,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0.3636,0.132205,138.9661,11.850000,31.973074,-0.326855,0.202569,4.563459,14.040421,497.131970,...,0.767598,0.152839,0.455611,0.377196,0.437523,29.375999,164.436509,430.436141,0.651397,1.270330
130,-0.3540,0.125316,124.2705,11.850000,31.972073,-0.326866,0.204219,3.984556,12.441325,554.079784,...,0.574784,0.364984,0.561717,0.357159,0.480373,24.895532,165.105971,384.973632,0.409651,1.399248
131,-0.4442,0.197314,124.3983,11.850000,31.972073,-0.326871,0.204315,3.984551,12.439904,464.821377,...,0.570497,0.368969,0.550420,0.363275,0.480499,24.939796,166.299510,388.900703,0.409199,1.394195
132,-0.8526,0.726927,130.9140,11.996133,31.972073,-0.377504,0.204290,3.984572,12.458502,589.900158,...,0.576007,0.336061,0.533630,0.431886,0.421620,22.937123,144.035418,372.377249,0.368102,1.387136


In [39]:
#cek jumlah kolom
dataset.shape

(134, 2980)

In [40]:
dataset.describe()

,ALogP,ALogp2,AMR,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,BCUTp-1l,BCUTp-1h,PPSA-1,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
count,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,...,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000
mean,-0.810360,1.235076,115.580041,11.935586,23.379450,-0.337141,0.281557,3.987857,10.801517,494.346691,...,0.717800,0.212123,0.549795,0.466040,0.428587,25.168249,126.365527,294.016663,0.585223,1.444422
std,0.763374,1.488011,17.183761,0.063883,6.186326,0.021821,0.079604,0.239195,1.439180,67.824423,...,0.115714,0.096722,0.049504,0.060183,0.055001,5.911029,47.022240,117.369536,0.160658,0.107387
min,-2.577400,0.000392,77.237500,11.850000,19.000412,-0.423178,0.202442,3.155671,9.268863,311.948533,...,0.458281,0.049935,0.378461,0.301073,0.282562,13.169813,43.726934,87.066620,0.273822,1.200308
25%,-1.277900,0.152253,101.864800,11.850000,19.000417,-0.335441,0.205341,3.983710,9.867517,452.574549,...,0.611305,0.140290,0.526534,0.427079,0.398992,20.939665,96.426108,224.525139,0.418849,1.360806
50%,-0.776600,0.610215,118.101400,11.988245,19.000421,-0.326876,0.284501,3.984051,10.082424,497.703576,...,0.756192,0.188997,0.551359,0.474677,0.433724,24.346414,118.221521,264.659598,0.634287,1.430805
75%,-0.320850,1.633028,129.522225,11.996136,31.972073,-0.326854,0.321664,3.984758,11.558583,541.536591,...,0.808424,0.289137,0.577552,0.508022,0.467439,29.392884,150.876289,356.123954,0.712636,1.516792
max,1.354100,6.642991,153.861900,11.999000,34.968855,-0.326828,0.529418,4.563459,14.040421,643.791922,...,0.911806,0.446848,0.677421,0.595420,0.601739,42.683693,278.796848,771.668789,0.867709,1.723927


In [41]:
# drop kolom yang ada nan
dataset.dropna(axis=1, inplace=True)

In [42]:
# drop kolom yang ada duplikat
dataset = dataset.loc[:,~dataset.columns.duplicated()]

In [43]:
# drop kolom yang ada 0
dataset = dataset[(dataset.sum(axis=1) != 0)]

In [44]:
#cek jumlah kolom
dataset.shape

(134, 2834)

In [45]:
X = dataset
y = pIC50_target

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)

(107, 2834)
(27, 2834)


In [47]:
#print datatest dan datatrain
X_train.to_csv('data_TA/test.csv', encoding='utf-8')
X_test.to_csv('data_TA/train.csv', encoding='utf-8')

In [48]:
X_train.isna().values.any()

False

In [49]:
X_test.isna().values.any()

False

In [50]:
X_train.shape

(107, 2834)

In [51]:
X_test.shape

(27, 2834)

In [52]:
#column similar value
# print(X_train[['ALogP']])

# arrTemp = X_train[[X_train.columns[0]]]
# for i in range(0, len(X_train.columns)):
#     same = 0
#     for j in range(0, len(X_train[[X_train.columns[i]]])):
#         if arrTemp.iloc[j].equals(X_train[[X_train.columns[i]]].iloc[j]):
#             same += 1
#     if (same / 107) * 100 >= 50:
#         print('>= 50 persen')

for i in X_train.columns:
    if (i != 'flag'):
        unique,counts = np.unique(X_train[i], return_counts=True)
        common_value = max(counts)
        similarity = common_value/len(X_train)*100
        temp_dev  = np.std(X_train[i].astype(float))
        if similarity >= 50:
            print('Drop column: ',i)
            X_train.drop(i,axis=1,inplace=True)
        
# print(dict(zip(unique,counts)))
# print('common_value: ',common_value)
# print('similarity : ',similarity)
# print('temp_dev : ',temp_dev)
X_train.shape

Drop column:  nAcid
Drop column:  naAromAtom
Drop column:  nAromBond
Drop column:  nBase
Drop column:  nB
Drop column:  C1SP1
Drop column:  C2SP1
Drop column:  C3SP3
Drop column:  C4SP3
Drop column:  SCH-3
Drop column:  SCH-4
Drop column:  SCH-5
Drop column:  VCH-3
Drop column:  VCH-4
Drop column:  SC-4
Drop column:  SC-6
Drop column:  VC-4
Drop column:  VC-6
Drop column:  nHBDon
Drop column:  khs.sLi
Drop column:  khs.ssBe
Drop column:  khs.ssssBe
Drop column:  khs.ssBH
Drop column:  khs.sssB
Drop column:  khs.ssssB
Drop column:  khs.sCH3
Drop column:  khs.dCH2
Drop column:  khs.tCH
Drop column:  khs.dsCH
Drop column:  khs.sssCH
Drop column:  khs.ddC
Drop column:  khs.tsC
Drop column:  khs.aaaC
Drop column:  khs.ssssC
Drop column:  khs.sNH3
Drop column:  khs.sNH2
Drop column:  khs.ssNH2
Drop column:  khs.dNH
Drop column:  khs.ssNH
Drop column:  khs.aaNH
Drop column:  khs.tN
Drop column:  khs.sssNH
Drop column:  khs.dsN
Drop column:  khs.aaN
Drop column:  khs.ddsN
Drop column:  khs.aas

Drop column:  n6FaRing
Drop column:  n7FaRing
Drop column:  n8FaRing
Drop column:  n9FaRing
Drop column:  n10FaRing
Drop column:  n11FaRing
Drop column:  n12FaRing
Drop column:  nG12FaRing
Drop column:  nFaHRing
Drop column:  n4FaHRing
Drop column:  n5FaHRing
Drop column:  n6FaHRing
Drop column:  n7FaHRing
Drop column:  n8FaHRing
Drop column:  n9FaHRing
Drop column:  n10FaHRing
Drop column:  n11FaHRing
Drop column:  n12FaHRing
Drop column:  nG12FaHRing
Drop column:  nFARing
Drop column:  n4FARing
Drop column:  n5FARing
Drop column:  n6FARing
Drop column:  n7FARing
Drop column:  n8FARing
Drop column:  n9FARing
Drop column:  n10FARing
Drop column:  n11FARing
Drop column:  n12FARing
Drop column:  nG12FARing
Drop column:  nFAHRing
Drop column:  n4FAHRing
Drop column:  n5FAHRing
Drop column:  n6FAHRing
Drop column:  n7FAHRing
Drop column:  n8FAHRing
Drop column:  n9FAHRing
Drop column:  n10FAHRing
Drop column:  n11FAHRing
Drop column:  n12FAHRing
Drop column:  nG12FAHRing
Drop column:  Topo

Drop column:  nF8Ring
Drop column:  nF9Ring
Drop column:  nF10Ring
Drop column:  nF11Ring
Drop column:  nF12Ring
Drop column:  nFG12Ring
Drop column:  nTRing
Drop column:  nT4Ring
Drop column:  nT5Ring
Drop column:  nT6Ring
Drop column:  nT7Ring
Drop column:  nT8Ring
Drop column:  nT9Ring
Drop column:  nT10Ring
Drop column:  nT11Ring
Drop column:  nT12Ring
Drop column:  nTG12Ring
Drop column:  nHeteroRing
Drop column:  n3HeteroRing
Drop column:  n4HeteroRing
Drop column:  n5HeteroRing
Drop column:  n6HeteroRing
Drop column:  n7HeteroRing
Drop column:  n8HeteroRing
Drop column:  n9HeteroRing
Drop column:  n10HeteroRing
Drop column:  n11HeteroRing
Drop column:  n12HeteroRing
Drop column:  nG12HeteroRing
Drop column:  nFHeteroRing
Drop column:  nF4HeteroRing
Drop column:  nF5HeteroRing
Drop column:  nF6HeteroRing
Drop column:  nF7HeteroRing
Drop column:  nF8HeteroRing
Drop column:  nF9HeteroRing
Drop column:  nF10HeteroRing
Drop column:  nF11HeteroRing
Drop column:  nF12HeteroRing
Drop co

(107, 2001)

In [53]:
#column standard deviation
X_train.std()

ALogP         0.788641
ALogp2        1.540137
AMR          17.375308
BCUTw-1l      0.064682
BCUTw-1h      6.017152
               ...    
Ts            6.085396
As           47.584352
Vs          115.593912
Ks            0.162599
Ds            0.102260
Length: 2001, dtype: float64

In [54]:
deviation = 0.95

X_train.drop(X_train.std()[X_train.std() < deviation].index.values, axis=1)

,ALogp2,AMR,BCUTw-1h,BCUTp-1h,PPSA-1,PPSA-2,PPSA-3,PNSA-1,PNSA-2,PNSA-3,...,L1i,L2i,Ti,Ai,Vi,L1s,L2s,Ts,As,Vs
59,2.588559,93.5823,31.974832,11.233238,424.634876,977.313317,27.232095,278.094567,-640.045222,-56.741196,...,20.541643,1.500647,23.318904,58.965261,121.636770,21.087945,1.491691,23.831794,59.729973,122.950527
73,0.708291,129.4863,19.000421,10.600639,496.461567,1572.792277,38.472072,385.697887,-1221.892484,-88.361761,...,21.647496,3.720081,26.747907,115.546060,253.452506,22.395439,3.746228,27.451989,118.152424,255.538385
33,0.265122,105.9922,19.000417,9.822371,477.640155,1148.697953,33.409307,262.847772,-632.134242,-57.084061,...,9.335070,5.735291,17.805193,94.754243,258.980526,9.630503,5.715935,18.042552,96.423082,262.879539
30,5.480281,99.0110,19.002047,9.955229,497.003918,1228.027709,35.400816,258.274676,-638.160883,-55.145763,...,10.178691,5.129742,17.928701,92.326260,247.069808,10.318134,5.148468,18.034142,92.833709,247.262239
106,0.435996,133.4999,19.000421,10.110195,525.654621,1724.832192,32.151482,359.021577,-1178.058650,-75.017266,...,26.885373,6.695598,35.159127,233.009660,552.258364,26.740033,6.639676,34.909077,228.595003,535.035882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2.180052,93.1939,31.974832,11.275113,437.300960,938.690620,28.862414,263.076625,-564.708480,-54.883071,...,15.767387,2.566548,19.746382,66.363463,143.268388,16.407929,2.475900,20.241250,66.257692,141.643334
103,1.633028,120.4713,19.000420,10.100369,455.728168,1438.015553,35.493077,358.598615,-1131.530641,-76.964651,...,21.323076,6.726784,29.363605,180.286099,398.087738,21.632001,6.799163,29.687902,182.810010,397.338188
67,2.573137,99.1758,19.000416,9.833449,509.881306,1221.637238,28.164827,271.375944,-650.196339,-63.843065,...,16.544998,3.199263,21.097562,79.651744,172.382008,16.851285,3.212381,21.361283,80.167707,171.772586
117,0.793703,137.3834,31.973074,13.358937,566.262357,1341.991793,30.638786,386.193391,-915.244243,-53.416599,...,13.836514,9.233680,24.467309,159.993656,362.959061,13.815243,9.138245,24.267798,156.415065,346.610593


In [55]:
# -----------REMOVE FEATURE WITH LOW CORRELATION TO Y RESPONSE---------------------
label_ = X_train.columns.values
pic50 = X_train.iloc[:,-1]
corr_y = [pearsonr(X_train.iloc[:,i],pic50) for i in range(X_train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(X_train.shape[1])]
corr_lim = 0.20
hi_corr = []
for i in range(len(corr_y)):
    if corr_y[i] > corr_lim:
        hi_corr.append(i)
label_idx = label_[hi_corr]
X_train = X_train.loc[:,label_idx]
print("descriptor number after removing low correlation with target: {}".format(X_train.shape[1]))

descriptor number after removing low correlation with target: 1088


In [56]:
# -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
# re-calculate correlation with pic50
label_ = X_train.columns.values
corr_y = [pearsonr(X_train.iloc[:,i],pic50) for i in range(X_train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(X_train.shape[1])]
desc_num = X_train.shape[1]
# calculate correlation for each descriptor
corr_matrix = np.corrcoef(X_train.T)
corr_lim = 0.80
low_corr = np.arange(desc_num).tolist()
tmp = np.arange(desc_num).tolist()
for i in np.arange(desc_num):
    tmp.remove(i)
    for j in tmp:
        corr_ = np.abs(corr_matrix[i,j])
        if corr_ >= corr_lim:
            if corr_y[i] > corr_y[j]:
                if j in low_corr:
                    low_corr.remove(j)
            else:
                if i in low_corr:
                    low_corr.remove(i)
label_idx = label_[low_corr]
X_train = X_train.loc[:,label_idx]
print("descriptor number after removing high correlation descriptor: {}".format(X_train.shape[1]))

descriptor number after removing high correlation descriptor: 100


In [57]:
label_ = X_train.columns.values
pickle.dump(label_, open('data_TA/Label.p','wb'))